# Amazon Bedrock Quickstart

This notebook demonstrates basic usage of Amazon Bedrock for:
- Listing available foundation models
- Text generation with Claude
- Embeddings with Titan
- Using the Converse API

In [ ]:
import os
import json
import boto3
from botocore.config import Config

# Set region
os.environ["AWS_REGION"] = "us-east-1"

config = Config(
    region_name=os.getenv("AWS_REGION"),
    retries={"max_attempts": 3, "mode": "adaptive"}
)

bedrock = boto3.client("bedrock", config=config)
bedrock_runtime = boto3.client("bedrock-runtime", config=config)

print(f"✓ Bedrock clients initialized for region: {os.getenv('AWS_REGION')}")

## 1. List Available Foundation Models

In [ ]:
response = bedrock.list_foundation_models()

print("Available Foundation Models:\n")
for model in response["modelSummaries"][:10]:  # Show first 10
    print(f"• {model['modelId']}")
    print(f"  Provider: {model['providerName']}")
    print(f"  Modalities: {', '.join(model.get('inputModalities', []))}")
    print()

## 2. Text Generation with Claude 3 Sonnet

In [ ]:
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
prompt = "Explain Amazon SageMaker in 2 sentences."

body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ]
})

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    body=body
)

response_body = json.loads(response["body"].read())
text = response_body["content"][0]["text"]

print(f"Prompt: {prompt}\n")
print(f"Response:\n{text}")

## 3. Generate Embeddings with Titan

In [ ]:
model_id = "amazon.titan-embed-text-v2:0"
text = "Amazon Bedrock provides foundation models via API"

body = json.dumps({
    "inputText": text
})

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    body=body
)

response_body = json.loads(response["body"].read())
embeddings = response_body["embedding"]

print(f"Text: {text}")
print(f"Embedding dimensions: {len(embeddings)}")
print(f"First 10 values: {embeddings[:10]}")

## 4. Converse API - Unified Interface

In [ ]:
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

response = bedrock_runtime.converse(
    modelId=model_id,
    messages=[
        {
            "role": "user",
            "content": [{"text": "What are the benefits of serverless architecture? Be concise."}]
        }
    ],
    inferenceConfig={
        "maxTokens": 512,
        "temperature": 0.7,
        "topP": 0.9
    }
)

assistant_message = response["output"]["message"]["content"][0]["text"]
print(f"Response:\n{assistant_message}")

## 5. Multi-turn Conversation

In [ ]:
conversation = []

# Turn 1
conversation.append({
    "role": "user",
    "content": [{"text": "What is AWS Lambda?"}]
})

response = bedrock_runtime.converse(
    modelId=model_id,
    messages=conversation,
    inferenceConfig={"maxTokens": 256, "temperature": 0.7}
)

assistant_msg_1 = response["output"]["message"]["content"][0]["text"]
conversation.append({
    "role": "assistant",
    "content": [{"text": assistant_msg_1}]
})

print("Turn 1")
print(f"User: What is AWS Lambda?")
print(f"Assistant: {assistant_msg_1}\n")

# Turn 2
conversation.append({
    "role": "user",
    "content": [{"text": "How does it compare to EC2?"}]
})

response = bedrock_runtime.converse(
    modelId=model_id,
    messages=conversation,
    inferenceConfig={"maxTokens": 256, "temperature": 0.7}
)

assistant_msg_2 = response["output"]["message"]["content"][0]["text"]
print("Turn 2")
print(f"User: How does it compare to EC2?")
print(f"Assistant: {assistant_msg_2}")

## Next Steps

- Explore different models and parameters
- Implement RAG (Retrieval Augmented Generation) with embeddings
- Use LangChain for more complex workflows (see `langchain_bedrock_example.py`)
- Check model access in the Bedrock console if you encounter permission errors